# Teorema Chino del Resto — **Avanzado** (SageMath)

Este cuaderno extiende el CRT más allá del caso básico de módulos coprimos. Incluye:

1. **Módulos no coprimos**: criterio de consistencia y solución única módulo `lcm`.
2. **Idempotentes canónicos** para recomposición rápida cuando los módulos son coprimos.
3. **Representación *mixed‑radix*** y **Algoritmo de Garner** (útil en RNS).
4. **Lifting de Hensel (lineal)** para subir soluciones de `p` a `p^k`.
5. **Aplicación práctica**: aceleración de RSA con CRT.
6. **CRT en polinomios** (en `GF(p)[x]`).

> **Kernel**: Seleccioná **SageMath** si Jupyter lo solicita.


In [ ]:
# Comprobación de entorno
version(), sys.version


## 1) CRT **general** con módulos no coprimos

Para

\[
x \equiv a_1 \pmod{m_1},\quad
x \equiv a_2 \pmod{m_2},
\]

hay solución **ssi** \(a_1 \equiv a_2 \pmod{g}\) con \(g=\gcd(m_1,m_2)\).
Si existe, es **única módulo** \(\operatorname{lcm}(m_1,m_2)\).


In [ ]:
def crt_pair_general(a1, m1, a2, m2):
    # Resuelve x ≡ a1 (mod m1) y x ≡ a2 (mod m2) con m1, m2 arbitrarios.
    # Devuelve (x0, M) con x ≡ x0 (mod M), M = lcm(m1, m2).
    # Lanza ValueError si no hay solución.
    a1, m1, a2, m2 = ZZ(a1), ZZ(m1), ZZ(a2), ZZ(m2)
    g = gcd(m1, m2)
    if (a2 - a1) % g != 0:
        raise ValueError(f"No hay solución: {a1} no es ≡ {a2} (mod {g})")
    m1p, m2p = m1 // g, m2 // g  # cocientes coprimos
    rhs = (a2 - a1) // g
    t = (rhs * inverse_mod(m1p % m2p, m2p)) % m2p  # m1' * t ≡ rhs (mod m2')
    x0 = a1 + m1 * t
    M = lcm(m1, m2)
    return (x0 % M, M)

def crt_general(residuos, modulos):
    # Combina x ≡ r_i (mod m_i) encadenando pares.
    # Verifica consistencia paso a paso y devuelve (x0, M) con M = lcm(m_i).
    if len(residuos) != len(modulos):
        raise ValueError("Longitudes no coinciden")
    x, M = ZZ(residuos[0]), ZZ(modulos[0])
    for r, m in zip(residuos[1:], modulos[1:]):
        x, M = crt_pair_general(x, M, ZZ(r), ZZ(m))
    return x, M

# Ejemplo consistente (módulos no coprimos): gcd(6, 9) = 3
x0, M = crt_general([2, 5], [6, 9])
x0, M, x0 % 6, x0 % 9


**Tests rápidos**


In [ ]:
# Caso sin solución
try:
    crt_pair_general(2, 6, 4, 9)  # 2 ≡? 4 (mod 3) → no
except ValueError as e:
    print("OK (sin solución):", e)

# Multimódulo (con mezcla de coprimos/no coprimos)
x0, M = crt_general([1, 3, 2], [8, 12, 9])
x0, M, [x0 % m for m in [8, 12, 9]]


## 2) Idempotentes canónicos (módulos coprimos)

Si \(n=\prod m_i\) con \(m_i\) **coprimos** dos a dos, existen idempotentes \(e_i\) tales que
\(e_i\equiv 1\pmod{m_i}\) y \(e_i\equiv 0\) módulo los otros. Entonces
\(x \equiv \sum_i r_i \, e_i \pmod{n}\).


In [ ]:
def crt_idempotents(mods):
    # mods: lista de módulos coprimos dos a dos.
    # Devuelve (n, [e_i]) idempotentes canónicos modulo n=prod(mods).
    mods = list(map(ZZ, mods))
    n = prod(mods)
    es = []
    for mi in mods:
        Ni = n // mi
        g, ui, _ = xgcd(Ni, mi)  # ui es inverso de Ni mod mi (g=1)
        ei = (Ni * ui) % n
        es.append(ei)
    return n, es

mods = [7, 11, 13]
n, es = crt_idempotents(mods)
# Verificación patrón 1/0
[[ei % m for m in mods] for ei in es]


In [ ]:
# Reconstrucción con idempotentes
res = [3, 10, 5]
x = sum(r*e for r, e in zip(res, es)) % n
x, [x % m for m in mods]


## 3) Algoritmo de **Garner** (mixed‑radix), módulos coprimos

Reescribe el número como
\(x = c_1 + c_2 m_1 + c_3 m_1m_2 + \cdots\)
a partir de residuos \((r_i \bmod m_i)\), evitando números grandes intermedios (útil en **RNS**).


In [ ]:
def garner(residuos, modulos):
    # Devuelve coeficientes c_i (mixed-radix) tales que
    #   x = c0 + c1*m0 + c2*m0*m1 + ...
    # Requiere módulos coprimos y residuos dados mod m_i.
    r = list(map(ZZ, residuos))
    m = list(map(ZZ, modulos))
    k = len(m)
    C = [ZZ(0)] * k   # "constantes" acumuladas por módulo
    A = [ZZ(1)] * k   # "coeficientes" acumulados por módulo
    c = [ZZ(0)] * k   # coeficientes mixed-radix (salida)
    for i in range(k):
        t = ((r[i] - C[i]) * inverse_mod(A[i] % m[i], m[i])) % m[i]
        c[i] = t
        for j in range(i+1, k):
            C[j] = (C[j] + A[j]*t) % m[j]
            A[j] = (A[j] * m[i]) % m[j]
    return c

def garner_reconstruct(c, m):
    x = ZZ(0)
    base = ZZ(1)
    for ci, mi in zip(c, m):
        x += ci * base
        base *= mi
    return x

mods = [7, 11, 13]
res = [3, 10, 5]
c = garner(res, mods)
x = garner_reconstruct(c, mods)
x, c, [x % mi for mi in mods]


## 4) **Hensel lifting (lineal)**: de \(p\) a \(p^k\)

Para la congruencia lineal \(a x \equiv b \pmod{p^k}\) con \(p\) primo y \(\gcd(a,p)=1\),
si conocemos \(x_1\) tal que \(a x_1 \equiv b \pmod p\), podemos **elevar** la solución a potencias superiores de \(p\).


In [ ]:
def hensel_lift_linear(a, b, p, k):
    # Resuelve a*x ≡ b (mod p^k), con p primo y gcd(a,p)=1.
    a, b, p, k = ZZ(a), ZZ(b), ZZ(p), ZZ(k)
    assert is_prime(p) and gcd(a, p) == 1
    # Nivel base (mod p)
    x = (b * inverse_mod(a, p)) % p
    mod = p
    for _ in range(1, k):
        # x actual resuelve mod p^t. Buscamos x' = x + t*p^t tal que a*x' ≡ b (mod p^{t+1}).
        # (b - a*x) es múltiplo de p^t; dividimos por p^t y resolvemos mod p.
        s = ((b - a*x) // mod) % p
        t_corr = (s * inverse_mod(a % p, p)) % p
        x = x + t_corr * mod
        mod *= p
    return x % mod

# Ejemplo: resolver 7*x ≡ 3 (mod 5^4)
hensel_lift_linear(7, 3, 5, 4)


## 5) Aplicación: **RSA con CRT** (descifrado rápido)

Si \(n=pq\) con \(p, q\) primos:

- \(d_p \equiv d \pmod{p-1}\), \(d_q \equiv d \pmod{q-1}\).
- \(m_p \equiv c^{d_p} \pmod p\), \(m_q \equiv c^{d_q} \pmod q\).
- Recombinar \(m\) a módulo \(n\) con CRT (idempotentes o `crt()` de enteros). 

Esto reduce exponenciaciones grandes a dos más chicas.


In [ ]:
# Demo RSA-CRT de juguete
p, q = 101, 113
n = p*q
phi = (p-1)*(q-1)
e = 17
d = inverse_mod(e, phi)

# Mensaje/cifrado
m = 12345 % n
c = power_mod(m, e, n)

# Descifrado directo
m_direct = power_mod(c, d, n)

# Descifrado con CRT
dp, dq = d % (p-1), d % (q-1)
mp = power_mod(c % p, dp, p)
mq = power_mod(c % q, dq, q)
# recombinar
x = crt(mp, mq, p, q)  # módulos coprimos
m_crt = x % n

m, m_direct, m_crt


## 6) CRT en **polinomios** sobre \(GF(p)\)

Si \(f_1, f_2\in GF(p)[x]\) son coprimos, vale
\(GF(p)[x]/(f_1f_2) \cong GF(p)[x]/(f_1) \times GF(p)[x]/(f_2)\).
La recombinación se hace igual que en enteros usando idempotentes con `xgcd`.


In [ ]:
# CRT para polinomios: combinación explícita por idempotentes
p = 101
R.<x> = GF(p)[]
m1 = x^2 + 1
m2 = x^2 + x + 2
assert gcd(m1, m2) == 1

M = m1*m2
E1 = M // m1
g1, U1, V1 = xgcd(E1, m1)   # U1 * E1 + V1 * m1 = 1  ⇒ U1 es inv de E1 mod m1
e1 = (E1 * U1) % M
E2 = M // m2
g2, U2, V2 = xgcd(E2, m2)
e2 = (E2 * U2) % M

# Residuos polinomiales
r1 = (x + 3) % m1
r2 = (2*x + 5) % m2

X = (r1*e1 + r2*e2) % M
# Verificación
(X % m1, X % m2)


---

### Resumen práctico
- **No coprimos**: verificar consistencia módulo `gcd`; solución única módulo `lcm`.
- **Coprimos**: usar **idempotentes** o **Garner** según convenga (reconstrucción directa vs. mixed‑radix/RNS).
- **Hensel**: sube soluciones a potencias de un primo con costo casi lineal en el exponente.
- **Aplicaciones**: RSA‑CRT, NTT/FFT modular, RNS, álgebra computacional (también en polinomios).

> Probá variar tamaños de módulos y medir tiempos con `%timeit` para ver la ganancia del CRT.
